In [1]:
from features import ensembleFeature
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import torch
from torch import nn
import re
from sklearn.utils import shuffle
from utils import metricsCal
from torch.utils.data import DataLoader,TensorDataset
import math
import sys
import copy
import pickle
from torch.autograd import Variable
from sklearn.model_selection import KFold
import torch.nn.functional as F
import os
import Attention_model as Model
from matplotlib import cm
from matplotlib.ticker import LinearLocator
import csv
%matplotlib inline

In [2]:
#一个函数用于序列转换，一个函数用于检测“N”碱基
def long_short(data):
    seq_list = []
    for i in data[0]:
        seq_list.append(str(i)[480:521])
    return np.array(seq_list)

def check_N(data1,data2):
    seq_list = []
    gene_list = []
    for i in range(len(data1)):
        if str(data1[i]).find("N")<0:
            seq_list.append(data1[i])
            gene_list.append(data2[i])
    return np.array(seq_list),np.array(gene_list)

In [3]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

In [4]:
# type_name = ["FullTranscript","matureRNA"]
# cell_name = ["A549","CD8T","Hek293_abacm","Hek293_sysy","HeLa","MOLM13"]
type_name = ["matureRNA"]
cell_name = ["MOLM13"]

network_name = ["BILSTM+Self-Attention"]

rows = len(type_name)*len(cell_name)*len(network_name)*5
row = -1
max_epochs = 20
max_patience = 20
batch_size = 20
row_name = []
Loss = np.zeros(rows*max_epochs).reshape(rows,max_epochs)
Acc = np.zeros(rows*(max_epochs+1)).reshape(rows,(max_epochs+1))
Mcc = np.zeros(rows*(max_epochs+1)).reshape(rows,(max_epochs+1))
Auc = np.zeros(rows*(max_epochs+1)).reshape(rows,(max_epochs+1))
Auc_test = np.zeros(rows*(max_epochs+1)).reshape(rows,(max_epochs+1))
th_All = np.zeros(rows*(max_epochs+1)).reshape(rows,(max_epochs+1))

In [5]:
Folder_name = []
for typei in range(len(type_name)):
    for celli in range(len(cell_name)):
        Folder_list = []
        trainPosSeq_file = "data/"+type_name[typei]+"/"+cell_name[celli]+"/Pos_"+cell_name[celli]+"_train_"+type_name[typei]+"_seq.csv"
        trainNegSeq_file = "data/"+type_name[typei]+"/"+cell_name[celli]+"/Neg_"+cell_name[celli]+"_train_"+type_name[typei]+"_seq.csv"
        testPosSeq_file = "data/"+type_name[typei]+"/"+cell_name[celli]+"/Pos_"+cell_name[celli]+"_test_"+type_name[typei]+"_seq.csv"
        testNegSeq_file = "data/"+type_name[typei]+"/"+cell_name[celli]+"/Neg_"+cell_name[celli]+"_test_"+type_name[typei]+"_seq.csv"
        trainPosGene_file = "data/"+type_name[typei]+"/"+cell_name[celli]+"/Pos_"+cell_name[celli]+"_train_"+type_name[typei]+"_gene.csv"
        trainNegGene_file = "data/"+type_name[typei]+"/"+cell_name[celli]+"/Neg_"+cell_name[celli]+"_train_"+type_name[typei]+"_gene.csv"
        testPosGene_file = "data/"+type_name[typei]+"/"+cell_name[celli]+"/Pos_"+cell_name[celli]+"_test_"+type_name[typei]+"_gene.csv"
        testNegGene_file = "data/"+type_name[typei]+"/"+cell_name[celli]+"/Neg_"+cell_name[celli]+"_test_"+type_name[typei]+"_gene.csv"
        
        #获取文件信息
        trainPos_seq = pd.read_csv(trainPosSeq_file,header=None)
        trainNeg_seq = pd.read_csv(trainNegSeq_file,header=None)
        testPos_seq = pd.read_csv(testPosSeq_file,header=None)
        testNeg_seq = pd.read_csv(testNegSeq_file,header=None)
        trainPos_gene = pd.read_csv(trainPosGene_file) 
        trainNeg_gene = pd.read_csv(trainNegGene_file)
        testPos_gene = pd.read_csv(testPosGene_file)
        testNeg_gene = pd.read_csv(testNegGene_file)
        #序列转41-nt长度
        trainPos_seq = long_short(trainPos_seq)
        trainNeg_seq = long_short(trainNeg_seq)
        testPos_seq = long_short(testPos_seq)
        testNeg_seq = long_short(testNeg_seq)
        #基因转35长度
        trainPos_gene = trainPos_gene.iloc[:,[0,1,3,4,5,6,7,8,9,10,11,13,14,16,17,19,23,22,25,20,21,44,45,46,47,48,49,50,65,66,69,55,51,57,56]]
        trainNeg_gene = trainNeg_gene.iloc[:,[0,1,3,4,5,6,7,8,9,10,11,13,14,16,17,19,23,22,25,20,21,44,45,46,47,48,49,50,65,66,69,55,51,57,56]]
        testPos_gene = testPos_gene.iloc[:,[0,1,3,4,5,6,7,8,9,10,11,13,14,16,17,19,23,22,25,20,21,44,45,46,47,48,49,50,65,66,69,55,51,57,56]]
        testNeg_gene = testNeg_gene.iloc[:,[0,1,3,4,5,6,7,8,9,10,11,13,14,16,17,19,23,22,25,20,21,44,45,46,47,48,49,50,65,66,69,55,51,57,56]]
        trainPos_seq,trainPos_gene = check_N(trainPos_seq,np.array(trainPos_gene))
        trainNeg_seq,trainNeg_gene = check_N(trainNeg_seq,np.array(trainNeg_gene))
        testPos_seq,testPos_gene = check_N(testPos_seq,np.array(testPos_gene))
        testNeg_seq,testNeg_gene = check_N(testNeg_seq,np.array(testNeg_gene))
        #基因变Float
        trainPos_gene = np.float32(trainPos_gene)
        trainNeg_gene = np.float32(trainNeg_gene)
        testPos_gene = np.float32(testPos_gene)
        testNeg_gene = np.float32(testNeg_gene)
        #打乱顺序，再取等长
        trainNeg_seq, trainNeg_gene = shuffle(trainNeg_seq,trainNeg_gene, random_state=1)
        testNeg_seq, testNeg_gene = shuffle(testNeg_seq, testNeg_gene, random_state=1)

#         trainNeg_seq = trainNeg_seq[:len(trainPos_seq)]
#         trainNeg_gene = trainNeg_gene[:len(trainPos_gene)]
#         testNeg_seq = testNeg_seq[:len(testPos_seq)]
#         testNeg_gene = testNeg_gene[:len(testPos_gene)]
        
        trainNeg_seq = trainNeg_seq[:500]
        testNeg_seq = testNeg_seq[:500]
        
        trainPos_seq = trainPos_seq[:500]
        testPos_seq = testPos_seq[:500]
        
#         trainNeg_seq = trainNeg_seq[:len(trainPos_seq)*10]
#         trainNeg_gene = trainNeg_gene[:len(trainPos_gene)*10]
#         testNeg_seq = testNeg_seq[:len(testPos_seq)]
#         testNeg_gene = testNeg_gene[:len(testPos_gene)]
#         trainPos_seq = trainPos_seq.repeat(10)
#         trainPos_gene = trainPos_gene.repeat(10)

        #第二重循环
        
        trainPos_emb = ensembleFeature.emb_seqs(trainPos_seq)
        trainNeg_emb = ensembleFeature.emb_seqs(trainNeg_seq)
        testPos_emb = ensembleFeature.emb_seqs(testPos_seq)
        testNeg_emb = ensembleFeature.emb_seqs(testNeg_seq)
            
        trainPos_PSNP,trainNeg_PSNP,testPos_PSNP,testNeg_PSNP = ensembleFeature.PSNP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq)
        
        trainPos_PCP = ensembleFeature.PCP(trainPos_seq)
        trainNeg_PCP = ensembleFeature.PCP(trainNeg_seq)
        testPos_PCP = ensembleFeature.PCP(testPos_seq)
        testNeg_PCP = ensembleFeature.PCP(testNeg_seq)
            
        trainPos_DBPF = ensembleFeature.DBPF(trainPos_seq)
        trainNeg_DBPF = ensembleFeature.DBPF(trainNeg_seq)
        testPos_DBPF = ensembleFeature.DBPF(testPos_seq)
        testNeg_DBPF = ensembleFeature.DBPF(testNeg_seq)
            
        trainPos = np.append(trainPos_emb.reshape(trainPos_seq.shape[0],1,-1),trainPos_PSNP.reshape(trainPos_seq.shape[0],1,-1),axis=2)
        trainPos = np.append(trainPos,trainPos_PCP.reshape(trainPos_seq.shape[0],1,-1),axis=2)
        trainPos = np.append(trainPos,trainPos_DBPF.reshape(trainPos_seq.shape[0],1,-1),axis=2)
        
        trainNeg = np.append(trainNeg_emb.reshape(trainNeg_seq.shape[0],1,-1),trainNeg_PSNP.reshape(trainNeg_seq.shape[0],1,-1),axis=2)
        trainNeg = np.append(trainNeg,trainNeg_PCP.reshape(trainNeg_seq.shape[0],1,-1),axis=2)
        trainNeg = np.append(trainNeg,trainNeg_DBPF.reshape(trainNeg_seq.shape[0],1,-1),axis=2)
        
        testPos = np.append(testPos_emb.reshape(testPos_seq.shape[0],1,-1),testPos_PSNP.reshape(testPos_seq.shape[0],1,-1),axis=2)
        testPos = np.append(testPos,testPos_PCP.reshape(testPos_seq.shape[0],1,-1),axis=2)
        testPos = np.append(testPos,testPos_DBPF.reshape(testPos_seq.shape[0],1,-1),axis=2)
        
        testNeg = np.append(testNeg_emb.reshape(testNeg_seq.shape[0],1,-1),testNeg_PSNP.reshape(testNeg_seq.shape[0],1,-1),axis=2)
        testNeg = np.append(testNeg,testNeg_PCP.reshape(testNeg_seq.shape[0],1,-1),axis=2)
        testNeg = np.append(testNeg,testNeg_DBPF.reshape(testNeg_seq.shape[0],1,-1),axis=2)
        
        

        trainPos_label = np.ones(len(trainPos))
        trainNeg_label = np.zeros(len(trainNeg))
        testPos_label = np.ones(len(testPos))
        testNeg_label = np.zeros(len(testNeg))

        trainData = np.append(trainPos,trainNeg,axis=0)
        testData = np.append(testPos,testNeg,axis = 0)
        trainLabel = np.append(trainPos_label,trainNeg_label,axis = 0)
        testLabel = np.append(testPos_label,testNeg_label,axis = 0)

        kf = KFold(5,True,10)
        #第三重循环
        for networki in range(len(network_name)):
            if network_name[networki] == "BILSTM+Self-Attention":
                for i,[train_index, test_index] in enumerate(kf.split(trainData)):
                    if i != 0:
                        continue
                    row+=1
                    row_one = type_name[typei]+"_"+cell_name[celli]+"_Early_Fusion_"+network_name[networki]+"_KFold_" + str(i)
                    row_name.append(row_one)
                    X_train = trainData[train_index]
                    X_test = trainData[test_index]
                    Y_train = trainLabel[train_index]
                    Y_test = trainLabel[test_index]
                    best_auc = 0
                    patience = 0
                    model_dir = "Model_Fusion_Extraction_Final/"+type_name[typei]+"_"+cell_name[celli]+"/"+\
                            "Early_Fusion"+"_"+network_name[networki]+"/KFold_" + str(i) + "/"
                    model = Model.ModelBS_Pro(trainData.shape[1],trainData.shape[2])
                    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  #选择设备
                    os.environ['CUDA_VISIBLE_DEVICES'] = '0'

                    for j in range(max_epochs):
                        runningLoss,th = Model.train(model,X_train,Y_train,i,device,model_dir,batch_size)
                        acc, mcc, auc=Model.test(X_test,Y_test,best_auc,device,model_dir,batch_size,th)
                        Loss[row][j] = runningLoss
                        th_All[row][j] = th
                        Acc[row][j] = acc
                        Mcc[row][j] = mcc
                        Auc[row][j] = auc

                        if auc > best_auc:
                            best_auc=auc
                            patience=0
                        else:
                            patience+=1
                        if patience>max_patience and best_auc!= 0.5:
                            break
                        
                        print("test_"+str(j))
                        _,_,auc_test=Model.independTest(testData,testLabel,device,model_dir,batch_size,0.5)
                        Auc_test[row][j] = auc_test

                    tmp_th = 0.5
                    tmp_auc = 0
                    for kk in range(max_epochs):
                        if Auc[row][kk]>tmp_auc:
                            tmp_th = th_All[row][kk]
                            tmp_auc = Auc[row][kk]
                    acc, mcc, auc=Model.independTest(testData,testLabel,device,model_dir,batch_size,tmp_th)
                    Acc[row][-1] = acc
                    Mcc[row][-1] = mcc
                    Auc[row][-1] = auc
                    th_All[row][-1] = tmp_th 

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


RuntimeError: CUDA out of memory. Tried to allocate 604.00 MiB (GPU 0; 11.91 GiB total capacity; 992.92 MiB already allocated; 249.94 MiB free; 996.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [10]:
a = torch.tensor(np.zeros(4400).reshape(100,44))
a[:,:40].shape

torch.Size([100, 40])

In [15]:
a[:,40:].reshape(a.shape[0],1,4)

torch.Size([100, 1, 4])